In [7]:
import pandas as pd
import requests

In [31]:
API_KEY = "QCC4AQUW1ZKX4T7P"
symbol = "TCS.BSE"

In [32]:
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=${symbol}&outputsize=full&apikey=${API_KEY}&datatype=csv"

In [20]:
response = requests.get(url)

In [21]:
# print response
if response.status_code == 200:
    with open('tcs_stock_data.csv', 'wb') as file:
        file.write(response.content)
    print("Data saved to tcs_stock_data.csv")


Data saved to tcs_stock_data.csv


In [36]:
# store the data in a pandas dataframe
df = pd.read_csv('tcs_stock_data.csv', index_col=0, parse_dates=True)

In [37]:
df.index = pd.to_datetime(df.index)
df = df.sort_index()
print(df.tail())

                 open       high        low      close  volume
timestamp                                                     
2025-06-30  3444.9500  3465.0000  3430.1499  3461.0500   80588
2025-07-01  3460.9500  3485.0000  3414.4500  3429.4500  221565
2025-07-02  3450.0500  3489.8501  3420.1001  3423.3501   54493
2025-07-03  3434.8999  3435.0000  3398.1499  3400.7500  113506
2025-07-04  3401.1001  3426.0000  3388.6499  3420.9500  170988


In [38]:
df.head()

,open,high,low,close,volume
timestamp,,,,,
2005-01-03,168.7250,169.7500,167.2814,169.1500,1185200
2005-01-04,168.8438,169.5000,166.7500,167.5438,1312656
2005-01-05,166.2500,167.2626,160.1250,165.6000,3669696
2005-01-06,165.8750,168.1250,161.5188,164.9876,3441640
2005-01-07,166.2500,168.0876,163.8876,167.4250,2627176


Call api for downloading RSI indicator data

In [33]:
rsi_url = f"https://www.alphavantage.co/query?function=RSI&symbol={symbol}&interval=daily&time_period=14&series_type=close&apikey={API_KEY}"
rsi_data = requests.get(rsi_url).json()

rsi_df = pd.DataFrame.from_dict(rsi_data['Technical Analysis: RSI'], orient='index')
rsi_df = rsi_df.rename(columns={'RSI': 'RSI'})
rsi_df.index = pd.to_datetime(rsi_df.index)
rsi_df = rsi_df.astype(float)


In [34]:
rsi_df.head()

,RSI
2005-01-24,13.8729
2005-01-25,20.8349
2005-01-27,22.9666
2005-01-28,40.9205
2005-01-31,43.9547


Merge RSI data with main df

In [40]:
# Align dates and merge
df = df.merge(rsi_df, how='left', left_index=True, right_index=True)
df.dropna(inplace=True)


In [50]:
df.head()

,open,high,low,close,volume,RSI_x,RSI_y,SMA,EMA
timestamp,,,,,,,,,
2005-01-24,155.6250,156.125,151.875,152.4500,2069208,13.8729,13.8729,70.3920,69.8612
2005-01-25,152.0000,154.750,150.625,154.3376,1968368,20.8349,20.8349,69.6661,69.1353
2005-01-27,155.0000,156.625,154.500,154.9376,1682024,22.9666,22.9666,69.1678,68.7777
2005-01-28,155.6250,161.875,155.500,161.2250,2935592,40.9205,40.9205,68.7073,68.5344
2005-01-31,161.8688,164.000,161.375,162.5814,2620200,43.9547,43.9547,68.5229,68.8273


Call api for downloading SMA indicator data and add it to main df

In [46]:
sma_url = f"https://www.alphavantage.co/query?function=SMA&symbol={symbol}&interval=daily&time_period=10&series_type=open&apikey={API_KEY}"
sma_data = requests.get(sma_url).json()
sma_df = pd.DataFrame.from_dict(sma_data['Technical Analysis: SMA'], orient='index')
sma_df = sma_df.astype(float)
sma_df.index = pd.to_datetime(sma_df.index)
df = df.merge(sma_df, how='left', left_index=True, right_index=True)
df.dropna(inplace=True)

In [49]:
ema_url = f"https://www.alphavantage.co/query?function=EMA&symbol={symbol}&interval=daily&time_period=10&series_type=open&apikey={API_KEY}"
ema_data = requests.get(ema_url).json()
ema_df = pd.DataFrame.from_dict(ema_data['Technical Analysis: EMA'], orient='index')
ema_df = ema_df.astype(float)
ema_df.index = pd.to_datetime(ema_df.index)
df = df.merge(ema_df, how='left', left_index=True, right_index=True)
df.dropna(inplace=True)